In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('./../data/movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1423,tt0410097,8000000,22201636,Hustle & Flow,Terrence Howard|Anthony Anderson|Taryn Manning...,Craig Brewer,The music will inspire them. The dream will un...,"With help from his friends, a Memphis pimp in ...",116,Drama,MTV Films,7/22/2005,6.2,2005
1054,tt1366365,20000000,16863583,The Cold Light of Day,Henry Cavill|VerÃ³nica Echegui|Sigourney Weave...,Mabrouk El Mechri,Be careful who you trust.,A young American (Cavill) uncovers a conspirac...,93,Action|Thriller,Summit Entertainment|Intrepid Pictures|Film Ri...,4/4/2012,4.8,2012
39,tt1618442,90000000,140396650,The Last Witch Hunter,Vin Diesel|Rose Leslie|Michael Caine|Elijah Wo...,Breck Eisner,Hunt forever.,"The modern world holds many secrets, but by fa...",106,Fantasy|Action|Adventure,Summit Entertainment|Atmosphere Entertainment ...,10/21/2015,5.6,2015
468,tt1341188,120000000,48668907,How Do You Know,Reese Witherspoon|Paul Rudd|Owen Wilson|Jack N...,James L. Brooks,How do you know it's love?,After being cut from the USA softball team and...,121,Comedy|Drama|Romance,Columbia Pictures|Gracie Films|Road Rebel,12/17/2010,4.9,2010
687,tt0814022,45000000,42487390,Bangkok Dangerous,Nicolas Cage|Shahkrit Yamnarm|Charlie Yeung|Pa...,Oxide Pang Chun|Danny Pang,There's only one way out.,"When carrying out a hit, assassin Joe (Cage) a...",99,Action|Crime|Thriller,Virtual Studios|Initial Entertainment Group|Sa...,8/21/2008,5.0,2008


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

data['profit'] = data.apply(lambda row: row.revenue - row.budget, axis=1)
data['release_date'] = pd.to_datetime(data['release_date'])
data['month'] = pd.DatetimeIndex(data['release_date']).month
data['genres'] = data['genres'].str.split('|')
data['production_companies'] = data['production_companies'].str.split('|')
data['cast'] = data['cast'].str.split('|')
data['director']  = data['director'].str.split('|')

data.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]","[Universal Studios, Amblin Entertainment, Lege...",2015-06-09,6.5,2015,1363528810,6
1,tt1392190,150000000,378436354,Mad Max: Fury Road,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",[George Miller],What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,"[Action, Adventure, Science Fiction, Thriller]","[Village Roadshow Pictures, Kennedy Miller Pro...",2015-05-13,7.1,2015,228436354,5
2,tt2908446,110000000,295238201,Insurgent,"[Shailene Woodley, Theo James, Kate Winslet, A...",[Robert Schwentke],One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,"[Adventure, Science Fiction, Thriller]","[Summit Entertainment, Mandeville Films, Red W...",2015-03-18,6.3,2015,185238201,3
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...",[J.J. Abrams],Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,"[Action, Adventure, Science Fiction, Fantasy]","[Lucasfilm, Truenorth Productions, Bad Robot]",2015-12-15,7.5,2015,1868178225,12
4,tt2820852,190000000,1506249360,Furious 7,"[Vin Diesel, Paul Walker, Jason Statham, Miche...",[James Wan],Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,"[Action, Crime, Thriller]","[Universal Pictures, Original Film, Media Righ...",2015-04-01,7.3,2015,1316249360,4


# Функции

In [5]:
def count_genres(genres):
    '''
    The function counts the number of genres
    
    Parameters:
    genres (array): array of genres
    '''
    for genre in genres:
        genres_counter[genre] += 1


def count_actor_revenue(row):
    '''
    The function counts the revenue for each artist
    
    Paramaters:
    row (Pandas' row)
    '''
    for actor in row['cast']:
        actor_counter[actor] += row['revenue']
        
        
def create_key(actors_pair):
    '''
    The function sorts tuple for creats sorted tuple like a key.
    
    Parameters:
    actors_pair (tuple): unsorted tuple
    
    Returns:
    tuple: sorted tuple like a key
    '''
    
    return tuple(sorted(actors_pair))

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
# тут пишем ваш код для решения данного вопроса:
data_result = data.sort_values(['budget'], ascending=False).head(1)[['imdb_id', 'original_title']]
data_result

,imdb_id,original_title
723,tt1298650,Pirates of the Caribbean: On Stranger Tides


ВАРИАНТ 2

In [7]:
data_result = data[data.budget == data['budget'].max()][['imdb_id', 'original_title']]

answers['1'] = f"{data_result['original_title'].values[0]} ({data_result['imdb_id'].values[0]})" # +

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
data_result = data.sort_values(['runtime'], ascending=False).head(1)[['imdb_id', 'original_title']]

answers['2'] = f"{data_result['original_title'].values[0]} ({data_result['imdb_id'].values[0]})" # +

# 3. Какой из фильмов самый короткий (в минутах)?





In [9]:
data_result = data.sort_values(['runtime']).head(1)[['imdb_id', 'original_title']]

answers['3'] = f"{data_result['original_title'].values[0]} ({data_result['imdb_id'].values[0]})" # +

# 4. Какова средняя длительность фильмов?


In [10]:
answers['4'] = round(data['runtime'].mean()) # +

# 5. Каково медианное значение длительности фильмов? 

In [11]:
answers['5']  = round(data['runtime'].median()) # +

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [12]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data_result = data.sort_values(['profit'], ascending=False).head(1)[['imdb_id', 'original_title']]

answers['6'] = f"{data_result['original_title'].values[0]} ({data_result['imdb_id'].values[0]})" # +

# 7. Какой фильм самый убыточный? 

In [13]:
data_result = data.sort_values(['profit']).head(1)[['imdb_id', 'original_title']]

answers['7'] = f"{data_result['original_title'].values[0]} ({data_result['imdb_id'].values[0]})" # +

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [14]:
answers['8'] = len(data.query('profit>0').index)

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [15]:
data_result = (data
                   .query('release_year==2008')
                   .sort_values(['revenue'], ascending=False)
                   .head(1)[['imdb_id', 'original_title']])

answers['9'] = f"{data_result['original_title'].values[0]} ({data_result['imdb_id'].values[0]})" # +

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [16]:
data_result = (data
                   .query('release_year >= 2012 & release_year <= 2014')
                   .sort_values(['profit'])
                   .head(1)[['imdb_id', 'original_title']])

answers['10'] = f"{data_result['original_title'].values[0]} ({data_result['imdb_id'].values[0]})" # +

# 11. Какого жанра фильмов больше всего?

In [17]:
genres_counter = Counter()

data['genres'].apply(count_genres)

answers['11'] = genres_counter.most_common()[0][0] # +

ВАРИАНТ 2

In [18]:
data.explode('genres')['genres'].value_counts().index[0]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [19]:
data_profitable = data[data['profit'] > 0][:]
answers['12'] = pd.Series(data_profitable['genres'].sum()).value_counts().index[0] # +

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [20]:
answers['13'] = (data
                     .explode('director').pivot_table(values=['profit'], index=['director'], aggfunc='sum')
                     .sort_values(['profit'], ascending=False)
                     .index[0]) # +

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [21]:
answers['14'] = (data
     .explode('genres')
     .explode('director')
     .query('genres=="Action"')
     .groupby(['director'])['director']
     .agg('count')
     .sort_values(ascending=False)
     .index[0]) # +

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [22]:
actor_counter = Counter()

data[data['release_year'] == 2012].apply(count_actor_revenue, axis=1)

answers['15'] = actor_counter.most_common()[0][0] # +

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [23]:
data_in_top = data[data['budget'] > data['budget'].mean()]
answers['16'] = pd.Series(data_in_top['cast'].sum()).value_counts().index[0] # +

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [24]:
nicolas_cage_data = data.explode('cast').query('cast=="Nicolas Cage"')
answers['17'] = pd.Series(nicolas_cage_data['genres'].sum()).value_counts().index[0] # +

# 18. Самый убыточный фильм от Paramount Pictures

In [25]:
pramount_pictures_data = data.explode('production_companies').query('production_companies=="Paramount Pictures"')

most_unprofitable_data = pramount_pictures_data[pramount_pictures_data['profit'] == pramount_pictures_data['profit'].min()]

answers['18'] = f"{most_unprofitable_data['original_title'].values[0]} ({most_unprofitable_data['imdb_id'].values[0]})" # +

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [26]:
answers['19'] = (data
                 .groupby(['release_year'])
                 .agg({'revenue': sum})
                 .sort_values('revenue', ascending=False)
                 .index[0]) # +

# 20. Какой самый прибыльный год для студии Warner Bros?

In [27]:
data_warner_bros = (data
                         .explode('production_companies')
                         .query('production_companies.str.contains("Warner Bros")', engine='python'))

answers['20'] = (data_warner_bros
                         .groupby(['release_year'])
                         .agg({'profit': sum})
                         .sort_values('profit', ascending=False)
                         .index[0]) # +

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [28]:
answers['21'] = data['month'].value_counts().index[0] # +

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [29]:
answers['22'] = len(data[data['month'].isin([6, 7, 8])].index) # +

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [30]:
answers['23'] = pd.Series(data[data['month'].isin([12, 1, 2])]['director'].sum()).value_counts().index[0] # +

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [31]:
data_prod_stud = data.copy()
data_prod_stud['title_len'] = data_prod_stud['original_title'].str.len()

answers['24'] = (data_prod_stud
     .explode('production_companies')
     .groupby('production_companies')
     .agg({'title_len': 'mean'})
     .sort_values(['title_len'], ascending=False)
     .index[0]) # +

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [32]:
data_prod_stud['overview_words_count'] = data_prod_stud['overview'].str.split().apply(len)

answers['25'] = (data_prod_stud
    .explode('production_companies')
    .groupby('production_companies')
    .agg({'overview_words_count': 'mean'})
    .sort_values(['overview_words_count'], ascending=False)
    .index[0]) # +

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [33]:
data.sort_values('vote_average', ascending=False).iloc[:(len(data.index)//100)]['original_title'].values

array(['The Dark Knight', 'Interstellar', 'The Imitation Game',
       'Inside Out', 'Room', 'The Wolf of Wall Street', 'Gone Girl',
       '12 Years a Slave', 'Guardians of the Galaxy',
       'The Lord of the Rings: The Return of the King', 'Memento',
       'Inception', 'The Pianist', 'The Grand Budapest Hotel', 'Her',
       'Spotlight', 'Big Hero 6', 'The Fault in Our Stars'], dtype=object)

In [34]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave' # +

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [35]:
comb_counter = Counter()

for actors in data['cast']:   
    for combined_pair in list(combinations(actors, 2)):
        key = create_key(combined_pair)
        comb_counter[key] += 1

most_common_together = comb_counter.most_common()[0][0]

answers['27'] = f"{most_common_together[0]} & {most_common_together[1]}" # +

ВАРИАНТ 2

In [36]:
comb_dict = {}

def count_combinations(artists):
    for combined_pair in list(combinations(artists, 2)):
        key = create_key(combined_pair)
        comb_dict.setdefault(key, 0)
        comb_dict[key] += 1

data['cast'].apply(count_combinations)

most_common_together = sorted(comb_dict.items(), key=lambda kv: kv[1], reverse=True)[0][0]

answers['27'] = f"{most_common_together[0]} & {most_common_together[1]}" # +

# Submission

In [37]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [38]:
# и убедиться что ни чего не пропустил)
len(answers)

27